## Q：汽车投诉信息采集：
#### 数据源：http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml
#### 投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态
#### 可以采用Python爬虫，或者第三方可视化工具
##### R：1、完成代码（20points）
#### 2、结果正确（20points）

# 使用request + BeautifulSoup提取12365auto投诉信息
## 1.导入模块

In [170]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import random

## 2.典型问题列表数据提取

In [171]:
#典型问题列表数据获取
js_url = "http://www.12365auto.com/js/cTypeInfo.js"
headers = {'user-agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
res = requests.get(js_url, headers = headers)

#典型问题数据提取
title_list = []
title = res.text

# 正则提取出json格式数据
patten = re.compile("\[.*\]")
temp = re.findall(patten, title)[0]

# 万能eval将json格式转为Python列表
temp = eval(temp)

for i in range(len(temp)):
    
    title_list.append([temp[i]["id"],temp[i]["name"],temp[i]["value"],temp[i]["zf"],temp[i]["items"]])
    
typical_infos_df = pd.DataFrame(title_list)
typical_infos_df.columns = ['id', 'name', 'value', 'zf', 'items']
display(typical_infos_df)

,id,name,value,zf,items
0,1,发动机,A,z,"[{'id': 9, 'title': '异响'}, {'id': 11, 'title':..."
1,2,变速器,B,z,"[{'id': 19, 'title': '顿挫'}, {'id': 20, 'title'..."
2,3,离合器,C,z,"[{'id': 16, 'title': '打滑'}, {'id': 17, 'title'..."
3,4,转向系统,D,z,"[{'id': 57, 'title': '失灵'}, {'id': 58, 'title'..."
4,5,制动系统,E,z,"[{'id': 29, 'title': '刹车失灵'}, {'id': 42, 'titl..."
5,6,轮胎,F,z,"[{'id': 10, 'title': '磨损'}, {'id': 39, 'title'..."
6,7,前后桥及悬挂系统,G,z,"[{'id': 25, 'title': '减震器漏油'}, {'id': 51, 'tit..."
7,8,车身附件及电器,H,z,"[{'id': 15, 'title': '车身异响'}, {'id': 21, 'titl..."
8,272,服务态度,I,f,"[{'id': 281, 'title': '互相推诿'}, {'id': 282, 'ti..."
9,273,人员技术,J,f,"[{'id': 298, 'title': '多次返修'}, {'id': 299, 'ti..."


## 3.爬虫工具实现

In [186]:
# 根据request_url得到soup
def get_page_content(request_url):
    
    # 得到页面的内容
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}
    html = requests.get(request_url, headers = headers, timeout = 10)
    content = html.text
    
    # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding = 'utf-8')
    
    return soup

#将【典型问题】的问题代码转换为中文
def parse_problems(typical_infos_df, value, item_id):

    items = typical_infos_df[typical_infos_df['value'] == value]['items'].iloc[0]
    items_list = []
    
    for j in range(len(items)):
        
        items_list.append([items[j]["id"],items[j]["title"]])
        
    problems_items_df = pd.DataFrame(items_list)
    problems_items_df.columns = ['id', 'title']

    problem_title = problems_items_df.query('id == %d' % item_id)['title'].iloc[0]

    return problem_title

# 分析当前页面的投诉
def analysis(soup):
    
    # 找到完整的投诉信息框
    temp = soup.find('div',class_ = "tslb_b")
    
    # 创建DataFrame
    temp_dict = []
    tr_list = temp.find_all('tr')
    
    for tr in tr_list[1:]:
        
        # ToDo：提取汽车投诉信息  
        ss = tr.find_all('td')
        problems_cate = ss[5].text
        problems_split = problems_cate.split(',')
        typical_problems = ''
        
        for s in problems_split:
           
            if s != '':
                try:
                    value, item_id = s[0], int(s[1:])
                    
                except ValueError:
                    value = s[0]
                    its = typical_infos_df[typical_infos_df['value'] == value]['name'].iloc[0]
                    typical_problems += its + ','
                    
                else:  
                    its = typical_infos_df[typical_infos_df['value'] == value]['name'].iloc[0]
                    title = parse_problems(typical_infos_df, value, item_id)
                    typical_problems += its + ':' + title + ','
                
        temp_dict.append([ss[0].text, ss[1].text, ss[2].text, ss[3].text, ss[4].text, ss[5].text, typical_problems, ss[6].text, ss[7].text])

    problems_items_df = pd.DataFrame(temp_dict)
    problems_items_df.columns=['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'typical_problems', 'datetime', 'status']

    return problems_items_df

## 4.数据爬取

In [187]:
#爬取前15页
page_num = 15
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'
num = 0
result = []

for i in range(page_num):
    
    request_url = base_url + str(i+1) + '.shtml'
    soup = get_page_content(request_url)
    print(request_url)
    
    df = analysis(soup)
    if num == 0:
        result = df
    else:
        result = pd.concat([result, df], axis = 0, ignore_index = True)
    num += 1
    time.sleep(random.randint(5, 10))
    
result = pd.DataFrame(result)
result = result.reset_index(drop = True)
display(result)

#爬取数据保存为csv文件
result.to_csv('car_complain.csv', index = False)

http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-1.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-2.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-3.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-4.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-5.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-6.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-7.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-8.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-9.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-10.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-11.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-12.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-13.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-14.shtml
http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-15.shtml


,id,brand,car_model,type,desc,problem,typical_problems,datetime,status
0,528094,一汽丰田,RAV4荣放,2019款 2.0L CVT 两驱 风尚X 限量版 国VI,一汽丰田RAV4荣放配件问题导致保险不理赔,"O354,","服务流程不完善:原车配套不全,",2020-09-30,信息审核
1,528091,奇瑞新能源,奇瑞eQ1,2019款 小蚂蚁 智享版 4座 30.6kWh,奇瑞eQ1 4S店不赠送充电桩 收取服务费,"K307,L313,","服务收费:变相收费,承诺不兑现:销售承诺不兑现,",2020-09-30,信息审核
2,528090,东风雪铁龙,雪铁龙C4L,2015款 1.2T 自动 豪华版,东风雪铁龙C4L方向盘晃动转向柱损坏,"D59,D80,","转向系统:异响,转向系统:抖动,",2020-09-30,信息审核
3,528089,东风乘用车,风神AX4,2018款 1.6L DCT 潮尚型 天窗版,东风风神AX4刹车盘生锈严重要求解决,"E42,","制动系统:生锈,",2020-09-30,厂家受理
4,528083,领克汽车,领克05,2020款 2.0TD 四驱 劲Halo,领克05自购车起无线充电功能无法使用,"H110,","车身附件及电器:电路故障,",2020-09-30,厂家受理
5,528081,一汽奔腾,奔腾T77,2020款 PRO 280 TID 自动 豪华型,一汽奔腾T77中控黑屏维修未解决,"H134,","车身附件及电器:影音系统故障,",2020-09-30,厂家受理
6,528080,东风日产,轩逸,2016款 1.6XV CVT 智尊版,东风日产轩逸仪表白屏 发动机噪音大,"A38,H202,H170,","发动机:噪音大,车身附件及电器:仪表故障,车身附件及电器:行车安全辅助系统故障,",2020-09-30,厂家受理
7,528078,长城汽车,哈弗H2,2014款 1.5T 手动 两驱精英版 5座,哈弗H2开空调发动机严重异响抖动,"A9,A24,","发动机:异响,发动机:抖动,",2020-09-30,厂家受理
8,528077,华晨鑫源,金杯T50,2020款 1.5L 创富型 后双轮 3.4米 国VI SWC15M,华晨金杯T50动力不足希望尽快处理,"A163,","发动机:功率不足,",2020-09-30,信息审核
9,528074,上汽大众,朗逸,2019款 两厢 280TSI DSG 豪华版 国VI,上汽大众朗逸韩泰轮胎出现两次爆胎,"F130,","轮胎:爆胎,",2020-09-30,信息审核


## 结果校验

In [190]:
# 校验下每一列是否有爬取遗漏。
result.isnull().any()

id                  False
brand               False
car_model           False
type                False
desc                False
problem             False
typical_problems    False
datetime            False
status              False
dtype: bool